CSV files need to be cleaned before inserting into SQL database in order to align them with accurate columns in database, and avoid datatype decrepencies

In [8]:
import pandas as pd
import numpy as np
from sqlalchemy import text
from sqlalchemy import create_engine

In [86]:
from dotenv import load_dotenv
import os

load_dotenv()

engine = create_engine(
    f"mysql+mysqlconnector://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
)


## Student Table

In [100]:
student_df=pd.read_csv('../03_Data/raw/students_data.csv')
student_df.head()


,Student_id,Name,Gender,Age,Contact,EmailID,State,Country,Signup_date,Active_status
0,S0001,Olivia,f,19,+1-555-348-4350,olivia@gmail.com,Florida,USA,16/01/2025,Active
1,S0002,James Walker,m,38,+1-555-200-8692,james.walker@gmail.com,New York,USA,2024-04-14,Active
2,S0003,Ava,f,24,+1-555-402-3497,ava@gmail.com,Texas,USA,2024-06-17,Active
3,S0004,Sophia Walker,M,44,+1-555-571-3756,sophia.walker@gmail.com,New York,USA,2025-01-11,Active
4,S0005,Olivia Wilson,Female,31,+1-555-882-1104,olivia.wilson@gmail.com,NaN,Canada,2023-07-31,Inactive


In [101]:
student_df.rename(columns={'Name': 'student_name', 'EmailID': 'email_id'}, inplace=True)

In [102]:
student_df.columns = student_df.columns.str.strip().str.lower()

In [103]:
student_df.head(1)

,student_id,student_name,gender,age,contact,email_id,state,country,signup_date,active_status
0,S0001,Olivia,f,19,+1-555-348-4350,olivia@gmail.com,Florida,USA,16/01/2025,Active


In [104]:
student_df=student_df[['student_id','student_name', 'age', 'gender', 'contact', 'email_id', 'state', 'country', 'signup_date', 'active_status']]

In [105]:
student_df.head(1)

,student_id,student_name,age,gender,contact,email_id,state,country,signup_date,active_status
0,S0001,Olivia,19,f,+1-555-348-4350,olivia@gmail.com,Florida,USA,16/01/2025,Active


In [106]:
student_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   student_id     1400 non-null   object
 1   student_name   1400 non-null   object
 2   age            1400 non-null   int64 
 3   gender         1400 non-null   object
 4   contact        1400 non-null   object
 5   email_id       1400 non-null   object
 6   state          1050 non-null   object
 7   country        1400 non-null   object
 8   signup_date    1400 non-null   object
 9   active_status  1400 non-null   object
dtypes: int64(1), object(9)
memory usage: 109.5+ KB


In [107]:
student_df.nunique()

student_id       1400
student_name      648
age                30
gender              7
contact          1400
email_id         1400
state               8
country             6
signup_date      1116
active_status       3
dtype: int64

In [108]:
student_df.isnull().sum()

student_id         0
student_name       0
age                0
gender             0
contact            0
email_id           0
state            350
country            0
signup_date        0
active_status      0
dtype: int64

In [109]:
student_df['gender'].unique()

array(['f', 'm', 'M', 'Female', 'F', 'Male', 'Prefer not to say'],
      dtype=object)

In [110]:
gender_map = {
    'f': 'Female',
    'F': 'Female',
    'Female': 'Female',
    'm': 'Male',
    'M': 'Male',
    'Male': 'Male',
    'Prefer not to say': 'Other'
}

student_df['gender'] = student_df['gender'].map(gender_map)

In [111]:
student_df['gender'].unique()   

array(['Female', 'Male', 'Other'], dtype=object)

In [112]:
student_df['active_status'].unique()

array(['Active', 'Inactive', 'Suspended'], dtype=object)

In [113]:
varchar_cols=student_df.select_dtypes(include='object').columns
student_df[varchar_cols].apply(lambda x: x.str.len().max())

student_id        5.0
student_name     18.0
gender            6.0
contact          15.0
email_id         27.0
state            11.0
country           9.0
signup_date      11.0
active_status     9.0
dtype: float64

In [114]:
student_df['signup_date'] = pd.to_datetime(student_df['signup_date'], dayfirst=True, format='mixed')
student_df['signup_date'] = student_df['signup_date'].dt.strftime('%Y-%m-%d')

In [115]:
student_df=student_df.to_sql(name='student_table', con=engine, if_exists='append', index=False)

IntegrityError: (mysql.connector.errors.IntegrityError) 1062 (23000): Duplicate entry 'S0001' for key 'student_table.PRIMARY'
[SQL: INSERT INTO student_table (student_id, student_name, age, gender, contact, email_id, state, country, signup_date, active_status) VALUES (%(student_id)s, %(student_name)s, %(age)s, %(gender)s, %(contact)s, %(email_id)s, %(state)s, %(country)s, %(signup_date)s, %(active_status)s)]
[parameters: [{'student_id': 'S0001', 'student_name': 'Olivia', 'age': 19, 'gender': 'Female', 'contact': '+1-555-348-4350', 'email_id': 'olivia@gmail.com', 'state': 'Florida', 'country': 'USA', 'signup_date': '2025-01-16', 'active_status': 'Active'}, {'student_id': 'S0002', 'student_name': 'James Walker', 'age': 38, 'gender': 'Male', 'contact': '+1-555-200-8692', 'email_id': 'james.walker@gmail.com', 'state': 'New York', 'country': 'USA', 'signup_date': '2024-04-14', 'active_status': 'Active'}, {'student_id': 'S0003', 'student_name': 'Ava', 'age': 24, 'gender': 'Female', 'contact'

## Instructor table

In [117]:
instructor_df=pd.read_csv('../03_Data/raw/instructors_data.csv')
instructor_df.head()

,Instructor_id,Instructor_name,Join_date,Email,Country,Active_status
0,INST001,Emma Anderson,02-09-2020,emma.anderson@gmail.com,USA,Active
1,INST002,ayaan walker,20-Mar-19,ayaan.walker@yahoo.com,USA,Inactive
2,INST003,Zara Iyer,01-10-2021,zara.iyer@teachwell.com,India,Active
3,INST004,Michael Smith,31-10-2023,michael.smith@edumaster.org,USA,Active
4,INST005,Sophia,12-03-2023,sophia.chatterjee@yahoo.com,India,Active


In [120]:
instructor_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Instructor_id    45 non-null     object
 1   Instructor_name  45 non-null     object
 2   Join_date        45 non-null     object
 3   Email            45 non-null     object
 4   Country          45 non-null     object
 5   Active_status    45 non-null     object
dtypes: object(6)
memory usage: 2.2+ KB


In [121]:
instructor_df.columns = instructor_df.columns.str.strip().str.lower()

In [122]:
instructor_df.rename(columns={'email': 'email_id'}, inplace=True)

In [123]:
instructor_df.nunique()

instructor_id      45
instructor_name    45
join_date          45
email_id           45
country             6
active_status       3
dtype: int64

In [124]:
instructor_df['active_status'].unique()

array(['Active', 'Inactive', 'On Hold'], dtype=object)

In [125]:
instructor_df['join_date'] = pd.to_datetime(instructor_df['join_date'], dayfirst=True, format='mixed')
instructor_df['join_date'] = instructor_df['join_date'].dt.strftime('%Y-%m-%d')

In [128]:
instructor_df=instructor_df.to_sql(name='instructor_table', con=engine, if_exists='append', index=False)

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1054 (42S22): Unknown column 'country' in 'field list'
[SQL: INSERT INTO instructor_table (instructor_id, instructor_name, join_date, email_id, country, active_status) VALUES (%(instructor_id)s, %(instructor_name)s, %(join_date)s, %(email_id)s, %(country)s, %(active_status)s)]
[parameters: [{'instructor_id': 'INST001', 'instructor_name': 'Emma Anderson', 'join_date': '2020-09-02', 'email_id': 'emma.anderson@gmail.com', 'country': 'USA', 'active_status': 'Active'}, {'instructor_id': 'INST002', 'instructor_name': 'ayaan walker', 'join_date': '2019-03-20', 'email_id': 'ayaan.walker@yahoo.com', 'country': 'USA', 'active_status': 'Inactive'}, {'instructor_id': 'INST003', 'instructor_name': 'Zara Iyer', 'join_date': '2021-10-01', 'email_id': 'zara.iyer@teachwell.com', 'country': 'India', 'active_status': 'Active'}, {'instructor_id': 'INST004', 'instructor_name': ' Michael Smith ', 'join_date': '2023-10-31', 'email_id': 'michael.smit

## Category table

In [2]:
category_df=pd.read_csv('../03_Data/raw/categories_data.csv')
category_df.head()

,category_id,category_name,parent_category_id
0,CAT001,Programming & Development,NaN
1,CAT002,Web Development,CAT001
2,CAT003,Mobile App Development,CAT001
3,CAT004,Data Science & Analytics,CAT001
4,CAT005,Database Management,CAT001


In [9]:
# Check CSV column names
print(category_df.columns.tolist())

# Check SQL table column names (pull directly from DB)
with engine.connect() as conn:
    result = conn.execute(text("DESCRIBE category_table"))
    sql_columns = [row[0] for row in result]
    print(sql_columns)

['category_id', 'category_name', 'parent_category_id']
['category_id', 'category_name', 'parent_category_id']


In [10]:
category_df.nunique()

category_id           22
category_name         22
parent_category_id     6
dtype: int64

In [11]:
category_df.to_sql(name='category_table', con=engine, if_exists='append', index=False)

22

## Course Table

In [15]:
course_df=pd.read_csv('../03_Data/raw/courses_data.csv')
course_df.head()

,course_id,course_name,instructor_id,category_id,price,creation_date,status
0,C0001,Complete Web Development Bootcamp,INST025,CAT002,3310,23-Feb-2023,Active
1,C0002,Vue.js Essential Training,INST007,CAT002,1299,06-May-2023,Active
2,C0003,web development for beginners,INST013,CAT002,2199,2023-11-11,active
3,C0004,Modern JavaScript Fundamentals,INST038,CAT002,2799,05/05/2023,inactive
4,C0005,Complete Web Development Bootcamp,INST006,CAT002,1037,2023-03-08,Active


In [17]:
# Check CSV column names
print(course_df.columns.tolist())

# Check SQL table column names (pull directly from DB)
with engine.connect() as conn:
    result = conn.execute(text("DESCRIBE course_table"))
    sql_columns = [row[0] for row in result]
    print(sql_columns)

['course_id', 'course_name', 'instructor_id', 'category_id', 'price', 'creation_date', 'status']
['course_id', 'course_name', 'instructor_id', 'category_id', 'price', 'creation_date', 'status']


In [18]:
course_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   course_id      160 non-null    object
 1   course_name    160 non-null    object
 2   instructor_id  160 non-null    object
 3   category_id    160 non-null    object
 4   price          160 non-null    int64 
 5   creation_date  160 non-null    object
 6   status         160 non-null    object
dtypes: int64(1), object(6)
memory usage: 8.9+ KB


In [22]:
# fetch existing instructor_ids from DB
existing_instructors = pd.read_sql("SELECT instructor_id FROM instructor_table", engine)
existing_categories = pd.read_sql("SELECT category_id FROM category_table", engine)

# check if any foreign keys in course_df are missing in parent tables
invalid_instructors = course_df[~course_df['instructor_id'].isin(existing_instructors['instructor_id'])]
invalid_categories = course_df[~course_df['category_id'].isin(existing_categories['category_id'])]

print("Invalid instructor_ids:", invalid_instructors['instructor_id'].unique())
print("Invalid category_ids:", invalid_categories['category_id'].unique())

Invalid instructor_ids: []
Invalid category_ids: []


In [23]:
course_df['creation_date'] = pd.to_datetime(course_df['creation_date'], format='mixed', dayfirst=True)
course_df['creation_date'] = course_df['creation_date'].dt.strftime('%Y-%m-%d')

In [24]:
course_df['status'].unique()

array(['Active', 'active', 'inactive', 'Inactive', 'INACTIVE', 'Removed',
       'ACTIVE', 'removed'], dtype=object)

In [25]:
status_map = {
    'Active': 'active',
    'active': 'active',
    'inactive': 'inactive',
    'Inactive': 'inactive',
    'INACTIVE': 'inactive',
    'Removed': 'removed',
    'ACTIVE': 'active',
    'removed': 'removed'
    }
course_df['status'] = course_df['status'].map(status_map)


In [26]:
course_df.to_sql(name='course_table', con=engine, if_exists='append', index=False)

160

## Enrollment Table

In [40]:
enrollment_df=pd.read_csv('../03_Data/raw/enrollments_data.csv')
enrollment_df.head()

,enrollment_id,student_id,course_id,enrollment_timestamp,discount_percentage,price,final_price
0,E00001,S0520,C0001,21/04/2023 14:45:05,0,3310,3310.00
1,E00002,S0277,C0001,2024-12-14 21:35:39,12,3310,2912.80
2,E00003,S0399,C0002,2024-10-17 20:46:53,22,1299,1013.22
3,E00004,S0283,C0002,2022-06-02 02:45,21,1299,1026.21
4,E00005,S1056,C0002,02-27-2022 16:24:49,0,1299,1299.00


In [41]:
# Check SQL table column names (pull directly from DB)
with engine.connect() as conn:
    result = conn.execute(text("DESCRIBE enrollment_table"))
    sql_columns = [row[0] for row in result]
    print(sql_columns)

['enrollment_id', 'student_id', 'course_id', 'enrollment_timestamp', 'discount', 'final_price']


In [42]:
enrollment_df.drop(columns=['price'], inplace=True)

In [43]:
enrollment_df.rename(columns={'discount_percentage': 'discount'}, inplace=True)

In [44]:
enrollment_df.columns = enrollment_df.columns.str.strip().str.lower()

In [45]:
enrollment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   enrollment_id         5000 non-null   object 
 1   student_id            5000 non-null   object 
 2   course_id             5000 non-null   object 
 3   enrollment_timestamp  5000 non-null   object 
 4   discount              5000 non-null   int64  
 5   final_price           5000 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 234.5+ KB


In [46]:
enrollment_df['enrollment_timestamp'] = pd.to_datetime(enrollment_df['enrollment_timestamp'], format='mixed', dayfirst=True)
enrollment_df['enrollment_timestamp'] = enrollment_df['enrollment_timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [47]:
enrollment_df['discount'] = pd.to_numeric(enrollment_df['discount'], errors='coerce')
enrollment_df['final_price'] = pd.to_numeric(enrollment_df['final_price'], errors='coerce')

In [48]:
print(enrollment_df.duplicated(subset=['enrollment_id']).sum())

0


In [49]:
existing_students = pd.read_sql("SELECT student_id FROM student_table", engine)
existing_courses = pd.read_sql("SELECT course_id FROM course_table", engine)

invalid_students = enrollment_df[~enrollment_df['student_id'].isin(existing_students['student_id'])]
invalid_courses = enrollment_df[~enrollment_df['course_id'].isin(existing_courses['course_id'])]

print("Invalid student_ids:", invalid_students['student_id'].unique())
print("Invalid course_ids:", invalid_courses['course_id'].unique())

Invalid student_ids: []
Invalid course_ids: []


In [50]:
enrollment_df.to_sql(name='enrollment_table', con=engine, if_exists='append', index=False)

5000

## Payments table


In [59]:
payment_df=pd.read_csv('../03_Data/raw/payments_data.csv')
payment_df.head()

,payment_id,enrollment_id,payment_date,amount_paid,payment_status
0,P0001,E00001,21/04/2023 14:50:07,3310.00,Successful
1,P0002,E00002,2024-12-14 21:44:48,2912.80,Successful
2,P0003,E00003,2024-10-17 20:48:04,1013.22,Successful
3,P0004,E00004,02/06/2022 02:53:32,1026.21,Successful
4,P0005,E00005,2022-02-27 16:28:46,1299.00,Successful


In [54]:
# Check SQL table column names (pull directly from DB)
with engine.connect() as conn:
    result = conn.execute(text("DESCRIBE payment_table"))
    sql_columns = [row[0] for row in result]
    print(sql_columns)

['payment_id', 'enrollment_id', 'payment_date', 'amount_paid', 'status']


In [55]:
payment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5600 entries, 0 to 5599
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   payment_id      5600 non-null   object 
 1   enrollment_id   5600 non-null   object 
 2   payment_date    5600 non-null   object 
 3   amount_paid     5600 non-null   float64
 4   payment_status  5600 non-null   object 
dtypes: float64(1), object(4)
memory usage: 218.9+ KB


In [76]:
payment_df.isnull().sum()

payment_id        0
enrollment_id     0
payment_date      0
amount_paid       0
status           35
dtype: int64

In [56]:
payment_df.nunique()

payment_id        5600
enrollment_id     5000
payment_date      5468
amount_paid       1600
payment_status       6
dtype: int64

In [63]:
payment_df.rename(columns={'payment_status': 'status'}, inplace=True)

In [60]:
payment_df['payment_date'] = pd.to_datetime(payment_df['payment_date'], format='mixed', dayfirst=True)
payment_df['payment_date'] = payment_df['payment_date'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [61]:
payment_df['amount_paid'] = pd.to_numeric(payment_df['amount_paid'], errors='coerce')
# verify no negative values due to CHECK constraint
print(payment_df[payment_df['amount_paid'] < 0])

Empty DataFrame
Columns: [payment_id, enrollment_id, payment_date, amount_paid, payment_status]
Index: []


In [65]:
payment_df['status'].unique()

array(['Successful', 'successful', 'Success', 'failed', 'Failed',
       'Failure'], dtype=object)

In [66]:
payment_df.columns = payment_df.columns.str.strip().str.lower()

In [74]:
print(payment_df['status'].isnull().sum())

35


In [81]:
print(payment_df[payment_df['status'].isnull()]['status'].unique())

[nan]


In [82]:
null_status_df = payment_df[payment_df['status'].isnull()]
print(null_status_df['enrollment_id'].duplicated().sum())
print(null_status_df['enrollment_id'].unique())

0
['E00971' 'E00625' 'E01127' 'E00576' 'E03141' 'E02689' 'E01432' 'E01581'
 'E03298' 'E00350' 'E02782' 'E00133' 'E02843' 'E04444' 'E00907' 'E00228'
 'E02560' 'E01139' 'E04679' 'E03657' 'E02710' 'E00760' 'E01773' 'E01587'
 'E02874' 'E01442' 'E04614' 'E03567' 'E03731' 'E02993' 'E01060' 'E04097'
 'E02032' 'E02937' 'E01607']


In [83]:
payment_df = payment_df.dropna(subset=['status'])

In [ ]:
status_map = {
    'Successful': 'Success',
    'successful': 'Success',
    'Success': 'Success',
    'failed': 'Failed',
    'Failed': 'Failed',
    'Failure': 'Failed'
}
payment_df['status'] = payment_df['status'].str.strip().map(status_map)

In [68]:
print(payment_df.duplicated(subset=['payment_id']).sum())

0


In [69]:
existing_enrollments = pd.read_sql("SELECT enrollment_id FROM enrollment_table", engine)
invalid_enrollments = payment_df[~payment_df['enrollment_id'].isin(existing_enrollments['enrollment_id'])]
print("Invalid enrollment_ids:", invalid_enrollments['enrollment_id'].unique())

Invalid enrollment_ids: []


In [84]:
payment_df.to_sql(name='payment_table', con=engine, if_exists='append', index=False)

5565

In [78]:
print(payment_df[payment_df['status'].isnull()]['status'].unique())

[nan]


In [80]:
null_status_df = payment_df[payment_df['status'].isnull()]
print(null_status_df['enrollment_id'].duplicated().sum())
print(null_status_df['enrollment_id'].unique())

0
['E00971' 'E00625' 'E01127' 'E00576' 'E03141' 'E02689' 'E01432' 'E01581'
 'E03298' 'E00350' 'E02782' 'E00133' 'E02843' 'E04444' 'E00907' 'E00228'
 'E02560' 'E01139' 'E04679' 'E03657' 'E02710' 'E00760' 'E01773' 'E01587'
 'E02874' 'E01442' 'E04614' 'E03567' 'E03731' 'E02993' 'E01060' 'E04097'
 'E02032' 'E02937' 'E01607']
